In [2]:
%run -i ./helpers.py

[nltk_data] Downloading package punkt to /Users/kamal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
tp = pd.read_pickle('./data/motley-fool-data.pkl')
tp['dt'] = tp.date.apply(lambda x: parse_dt(x))
tp.drop(columns = ['date','exchange'],inplace=True)

snp = pd.read_csv('./data/snp_constituents.csv')
snp.Symbol = snp.Symbol.astype(str)
constituents_snp = [x for x in list(tp.ticker.unique()) if x in snp.Symbol.tolist()]
print(f'we have {len(constituents_snp)} cons in S&P 500')
tp_snp = tp[tp.ticker.isin(constituents_snp)]

we have 474 cons in S&P 500


In [4]:
print(tp.ticker.nunique())
tp.dt.describe()

2876


count                   18375
unique                   5201
top       2021-11-04 17:00:00
freq                       73
first     2019-04-11 10:00:00
last      2023-02-23 11:00:00
Name: dt, dtype: object

In [3]:
# limit_tickers = ['DAL']
# tp = tp[tp.ticker.isin(limit_tickers)]

In [4]:
# tp[tp.ticker=='DAL'].to_csv('./dal.csv', index=None)

In [5]:
# tp[tp.ticker=='DAL'].iloc[0].transcript

In [7]:
def count_splits_transcripts(tr):
    qna = [x for x in sent_tokenize(tr) if 'questions and answers' in str.lower(x) or 'questions & answers' in str.lower(x)] 
    
    if(len(qna)>0):
        return 1
    else:
        qnaa = [x for x in sent_tokenize(tr) if 'analyst' in str.lower(x) or 'portfolio manager' in str.lower(x)] 
        if(len(qnaa)>0):
            return 2
        else:
            return 0
        
def split_transcript(row):
    splits = re.split(r'questions and answers|questions & answers', str.lower(row.transcript))
    if(len(splits)>0):
        row['prep_remarks'] = splits[0]
        row['QnA'] = ' '.join(splits[1:None])
    else:
        splits = re.split(r'analyst|portfolio manager', str.lower(row.transcript))
        if(len(splits)>0):
            row['prep_remarks'] = splits[0]
            row['QnA'] = ' '.join(splits[1:None])
        else:
            print('exception')
            row['prep_remarks'] = str.lower(row.transcript)
            row['QnA'] = str.lower(row.transcript)
    return(row)

In [8]:
tp_splits = tp.apply(lambda x:split_transcript(x), axis=1)
tp_splits.drop(columns='transcript',inplace=True)
print(tp_splits[tp_splits.QnA.isna()].shape)

(0, 5)


In [9]:
# tp_splits.to_csv('./data/tp_tics_splits.csv',index=None)
tp_splits.to_csv('./data/tp_tics_splits.csv',index=None)

In [10]:
tp_splits

,q,ticker,dt,prep_remarks,QnA
0,2020-Q2,BILI,2020-08-27 21:00:00,"prepared remarks:\noperator\ngood day, and wel...",":\noperator\nthank you, sir. [operator instruc..."
1,2020-Q3,GFF,2020-07-30 16:30:00,prepared remarks:\noperator\nthank you for sta...,:\noperator\ncertainly. [operator instructions...
2,2020-Q1,LRCX,2019-10-23 17:00:00,prepared remarks:\noperator\ngood day and welc...,:\noperator\nthank you. [operator instructions...
3,2019-Q3,BBSI,2019-11-06 12:00:00,"prepared remarks:\noperator\ngood day, everyon...",":\noperator\nthank you, sir. [operator instruc..."
4,2019-Q2,CSTE,2019-08-07 08:30:00,prepared remarks:\noperator\ngreetings and wel...,":\noperator\nthank you. at this time, we'll be..."
...,...,...,...,...,...
18750,2021-Q3,SWX,2021-11-09 13:00:00,prepared remarks:\noperator\nladies and gentle...,:\noperator\nthank you. [operator instructions...
18751,2021-Q4,PNNT,2021-11-18 12:00:00,"prepared remarks:\noperator\ngood morning, and...",:\noperator\nthank you. [operator instructions...
18752,2022-Q1,TDG,2022-02-08 11:00:00,prepared remarks:\noperator\nthank you for sta...,:\noperator\n[operator instructions] our first...
18753,2021-Q4,DVAX,2022-02-28 16:30:00,"prepared remarks:\noperator\ngood day, ladies ...",:\noperator\nthank you. [operator instructions...
